# Linear Regression
<div class="alert alert-block alert-info">
<b>Content:</b> In this notebook we run linear regression for a couple of experiments
    
* analysis of data
* prediction of data using the plain linear regression
* prediction of data using modified versions
</div>


In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.base import BaseEstimator
from sklearn.preprocessing import PolynomialFeatures

## The Dataset

The following uses a __hypothetical__ demo dataset provided by the company glassdoor. The dataset is suitable for analyzing gender gaps in populations, e.g. the employees of a company.

In [ ]:
raw_data=pd.read_csv("data/glassdoor.csv")
print(raw_data.shape)

In [ ]:
raw_data.head()

In [ ]:
target=raw_data.basePay+raw_data.bonus
raw_features =raw_data.drop(["basePay", "jobTitle", "bonus"], axis=1)
features=pd.get_dummies(raw_features, drop_first=True)

In [ ]:
features.head()

## Descriptive Data Analysis

### Checking for Correlation

In [ ]:
print("age:", pearsonr(target, features.age)) # returns correlation and p-value (smaller means more indication of a non-coincidental result)
print("perfEval:", pearsonr(target, features.perfEval))
print("seniority:", pearsonr(target, features.seniority))
print("gender_Male:", pearsonr(target, features.gender_Male))

In [ ]:
def pearson(i):
    a=pearsonr(target, features[i])
    return [a.statistic, a.pvalue]
    
d=pd.DataFrame(columns=["feature", "pearson", "p-value"], 
               data=[[i]+pearson(i) for i in features.columns])
d.round(2).sort_values("pearson")

* We observe significant high correlation between
    * target and age and
    * target and seniority.

There is also a (less strong yet significant) correlation between target and gender!

In [ ]:
plt.scatter(raw_data.age, target)

In [ ]:
plt.scatter(raw_data.perfEval, target)

In [ ]:
plt.scatter(raw_data.seniority, target)

Plots are nice, but often deceptive. Even though the two above plots look similar, the correlation they are supposed to show is absolutely not similar!

### Analysis By Linear Regression on the Full Dataset
Linear Regression as an Exploratory Tool

In [ ]:
reg_full= LinearRegression()
reg_full.fit(np.array(features), target)

In [ ]:
reg_full_r2=r2_score(target, reg_full.predict(np.array(features)))
reg_full_r2

The linear regression model explains some part (64%) of the variance. There is also some unexplained variance in the data.

In [ ]:
w=reg_full.coef_
w

In [ ]:
for i in range(0, len(features.columns)): 
    print(features.columns[i], w[i])

Interpretation Examples of the model: Ceteris Paribus
* aging a year correlates with a total pay increase of 693 dollars
* with high school as highest education (instead of college) decreases total pay by -2012 dollars, wheras a PhD increases the total py by 6463 dollars
* departments Engineering and Management lead to higher sallaries than Operations and Sales
* males receive 6352 dollars more than females

These numbers assume that the learned model fully describes the data and they are not to be misunderstood as mathematical properties!

The population parameters are NO measure for correlation (compare e.g. age and perfEval!)

The results say nothing about causation. Thus it would not be reasonable to assume that generally simply getting older increases your pay.

# Plotting Residuals

In [ ]:
plt.scatter(target, target-reg_full.predict(np.array(features)))
plt.xlabel("Target")
plt.ylabel("Residual")
plt.savefig('results/target-residuals.pdf')

In [ ]:
plt.scatter(features.age, target-reg_full.predict(np.array(features)))
plt.xlabel("Age")
plt.ylabel("Residual")
plt.savefig('results/residuals.pdf')

* The residuals plot reveals high residuals.
* This suggests that not enough information is given in the features to fully explain the target.
* There is no obvious dependence of residuals on age.

### Using Linear Regression to Investigate Specific Variables

In [ ]:
reg_age = LinearRegression()
reg_age.fit(np.expand_dims(np.array(raw_data.age), axis=1), target)
w=reg_age.coef_
b=reg_age.intercept_
age_space=np.arange(0,70,1)
y=reg_age.predict(age_space.reshape(-1,1))

In [ ]:
r2_score(target, reg_age.predict(np.array(raw_data[["age"]])))

There is (of cause) much more unexplained variance.

In [ ]:
plt.scatter(raw_data.age, target)
plt.plot(age_space, y, c='black')

### Investigate Two Variables
Equal-Slopes method (assumption: both features are independent)

In [ ]:
reg_age_sen = LinearRegression()
reg_age_sen.fit(np.array(raw_data[["age", "seniority"]]), target)

In [ ]:
reg_age_sen.coef_

In [ ]:
r2_score(target, reg_age_sen.predict(np.array(raw_data[["age", "seniority"]])))

In [ ]:
age_space=np.arange(0,70,1)
seniority_space=np.arange(1,6,1)

In [ ]:
plt.scatter(raw_data.age, target)
for s in range(0,len(seniority_space), 1):
    grid=np.stack((age_space, seniority_space[s]*np.ones_like(age_space)), axis=1)
    y=reg_age_sen.predict(grid)
    plt.plot(age_space, y, label=seniority_space[s])
plt.legend()

higher age and higher seniority correlate positively with higher total pay
Linear model -> parallel slopes. Alterantive: Compute own models for each seniority level (treat different seniorities as different populations)

# Evaluation of Machine Learning Models

In [ ]:
inner_cv = KFold(n_splits=10, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=10, shuffle=True, random_state=1)

def print_eval(result):
    #print(result, 3)
    print("RMSE: {:,.2f}".format(np.round(np.sqrt(-1*res.mean()), 3)))
    print(" MSE: {:,.2f}".format(-1*np.round(res.mean(), 3)), "±", "{:,.2f}".format(np.round(res.std(), 3)))

## Linear Regression

In [ ]:
res=cross_val_score(LinearRegression(), np.array(features), target, cv=outer_cv, n_jobs=5, scoring='neg_mean_squared_error')
print_eval(res)

## Polynomial Regression

In [ ]:
poly=PolynomialFeatures(degree=4)
X_poly=poly.fit_transform(np.array(features))
print(features.shape)
print(X_poly.shape)
reg=LinearRegression()
reg.fit(X_poly,target)

In [ ]:
r2_score(target, reg.predict(X_poly))

In [ ]:
reg_full_r2 # for comparison the respective value of the linear fit

The significantly(!) more complex model explains the variance better. But, does it lead to better predictive performance or rather to overfitting? -> Proper ML Evaluation

In [ ]:
class PolynomialRegression(BaseEstimator):
    def __init__(self, degree=1):
        self.degree = degree
    
    def fit(self, X, y):
        self.model = LinearRegression(fit_intercept=False) # polynomial features create a feature with degree 0: 1
        self.poly_feature=PolynomialFeatures(degree=self.degree)
        X_poly=self.poly_feature.fit_transform(X)
        self.model.fit(X_poly, y)
    
    def predict(self, X):
        return self.model.predict(self.poly_feature.transform(X))
    
    def coef_(self):
        return self.model.coef_


In [ ]:
poly_reg=PolynomialRegression()

poly_reg_grid={
    'degree':[2,3,4]
}
poly_reg_grid_cv = GridSearchCV(
    estimator=poly_reg, param_grid=poly_reg_grid, cv=inner_cv, n_jobs=5, scoring='neg_mean_squared_error')
res=cross_val_score(poly_reg_grid_cv, np.array(features), target, cv=outer_cv, n_jobs=5, scoring='neg_mean_squared_error')

print_eval(res)

In [ ]:
poly_reg_grid_cv.fit(features,target)
poly_reg_grid_cv.best_params_

* Interpretation of polynomial regression is much more difficult.
* However, additional non-linear variable allow a better fit to the data (more explained variance)
* HOWEVER: This did not translate into a boost of predictive performance!

<div class="alert alert-block alert-info">
<b>Take Aways:</b> 

* Running Linear Regression.
* Linear Regression Coefficients can be interpreted (ceteris paribus)
* The interpretation is however limited.
* The residuals can and should be investigated.
* Similar to the kernel trick of SVMs, we can turn linear regression into non-linear regression, here polynomial regression.
</div>

Remarks on the observed gender pay gap evidence:
* Data ignores non-binary gender.
* Data taken from https://research.glassdoor.com/site-us/wp-content/uploads/sites/2/2019/03/GD_Report_AnalyzingGenderPayGap_v2-2.pdf

* It represents __hypothetical__ data, thus __no real evidence__.

__If__ the data __was__ real, our analysis would have revealed, that a gender pay gap does exist -- within the considered data. (This is a topic with potentially vastly different results in different countries and different industries).

* Multiple factors might lead directly or indirectly to a pay gap
     * directly: individuals of a group get payed less, despite the same education/job-title/seniority/... 
     * indirectly:
         * a group works more often in less-well payed jobs
         * a group is underrepresented in the higher educational
         * a group does not reach the same seniority levels as others (usually seniority is correlated with long-term full-time employment)

To investigate such factors, one can gather first evidence:

In [ ]:
raw_data

In [ ]:
raw_data["total"] = raw_data.basePay+raw_data.bonus

In [ ]:
gender_avg=raw_data[['gender', 'total']].groupby('gender').aggregate(['min', 'max', 'mean'])
gender_avg

In [ ]:
def gap(a,b):
    return (b-a)/b*100

In [ ]:
gap(gender_avg[('total', 'mean')]['Female'], gender_avg[('total', 'mean')]['Male'])

In [ ]:
mean_per_jobtitle=raw_data[['jobTitle', 'total']].groupby(['jobTitle']).aggregate(mean=('total', 'mean'))
mean_per_jobtitle

In [ ]:
def get_gender_table_for_criterion(raw_data, criterion):
    mean_per_criterion=raw_data[[criterion, 'total']].groupby([criterion]).aggregate(mean=('total', 'mean'))

    a=raw_data[['gender', criterion, 'total']].groupby(['gender', criterion]).aggregate(mean=('total', 'mean'), count=('total', 'count'))
    a=a.reset_index()
    b=a.pivot(index=criterion, columns='gender')
    b.columns = ['_'.join(col) for col in b.columns]
    
    b['gap']=b.apply(lambda x: gap(a = x['mean_Female'], b = x['mean_Male']), axis=1)
    
    b = pd.concat([b,mean_per_criterion], axis=1) 
    b=b.sort_values('mean')
    return b

In [ ]:
get_gender_table_for_criterion(raw_data, 'jobTitle').round(2)

In [ ]:
get_gender_table_for_criterion(raw_data, 'seniority')

In [ ]:
get_gender_table_for_criterion(raw_data, 'edu')

__Technical Remarks___
All these investigations could be conducted by grouping by more than just one entity (e.g. combinations of edu/seniority/jobTitle/dept.
* Generally very many subgroups to investigate separately, no overall impression.
* Often small groups (statistical significance problematic).
* Other ideas include "corrections" - adjustments to account for the influcences of other groups.
* Uneven Distribution over different fields often leads to counter-intuitive results. Look up "Simpson's Paradox".

__General Remarks__
* No matter which influences create the pay gap (direkt or indirect influences), it is still a pay gap. The analysis of the factors does not change that, however, depending on the factors, different levers are needed to fix it.
* While this data is hypothetical, the gender pay gap is a well-known statistically proven phenomenon. E.g. a comparison on European countries here: https://www.destatis.de/Europa/EN/Topic/Population-Labour-Social-Issues/Labour-market/gender_pay_gap.html


<div class="alert alert-block alert-success">
<b>Play with:</b> 
    
* change the target attribute (e.g. square the values, use the logarithm)
* use linear regression on that and compare to polynomial regression
</div>
